# Importation des modules nécessaires

In [1]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from cleaner import clean_corpus
from weather import get_weather
from wikipediaprocess import search_wikipedia
from newsapi import NewsApiClient
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Initialisation de l'API de news

In [2]:
newsapi = NewsApiClient(api_key='a1d8789da28b4df282afbabff793c211')

# Initialisation du chatbot

In [3]:
chatbot = ChatBot("Chatbot tounsi",logic_adapters=[      
    # Adapter pour évaluer des expressions mathématiques
    {
        'import_path': 'chatterbot.logic.MathematicalEvaluation',
    },
    # Adapter pour trouver la meilleure réponse en fonction du contexte
    {
        'import_path': 'chatterbot.logic.BestMatch',
    }
])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mohamed-Amine-
[nltk_data]     Miladi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to C:\Users\Mohamed-Amine-
[nltk_data]     Miladi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Mohamed-Amine-
[nltk_data]     Miladi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Fichier contenant la conversation à utiliser pour entraîner le chatbot

In [4]:
CORPUS_FILE="chat2.txt"

# Entraînement du chatbot sur le fichier de corpus nettoyé

In [5]:
trainer = ListTrainer(chatbot)
cleaned_corpus=clean_corpus(CORPUS_FILE)
trainer.train(cleaned_corpus)

List Trainer: [####################] 100%


# Créer une liste de phrases nettoyées

In [6]:
sentences = list(cleaned_corpus[1:])

# Conditions pour quitter le chatbot

In [7]:
exit_conditions = (":q", "quit", "exit")

# Exécution du ChatBot

In [12]:
while True:
    # Affichage des instructions pour l'utilisateur
    print("*********************************************************\nPour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays\nPour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet\nPour obtenir la définition d'un sujet, tapez : wiki + sujet\nPour quitter, tapez : quit ou exit\n************************************************************")
    # Lecture de l'entrée utilisateur
    query = input("> ")
    # Vérification si l'utilisateur veut quitter
    if query in exit_conditions:
        break
    # Récupération de la météo pour un pays donné
    elif query.startswith("chnowa ta9s fi "):
        # Récupération du nom du pays
        country = query.replace("chnowa ta9s fi ", "")
        # Récupération de la météo pour le pays
        weather = get_weather(country)
        print(f"ta9s: {weather}")
    # Recherche de la définition d'un sujet donné sur Wikipedia
    elif query.startswith("wiki "):
        topic=query.replace("wiki ","")
        wiki=search_wikipedia(topic)
        print(wiki)
    # Récupération des dernières actualités sur un sujet donné
    elif query.startswith("e5er a5bar aala "):
        # Récupération du sujet
        theme=query.replace("e5er a5bar aala ","")
        # Récupération des dernières actualités
        top_headlines = newsapi.get_everything(q=theme, sort_by='relevancy',language='ar')
        for article in top_headlines['articles']:
            print(article['title'])
    # Si aucune des commandes ci-dessus n'a été reconnue, le chatbot tente de répondre
    else:
        #print(f"🤖 {str(chatbot.get_response(query))}")

        # Ajouter le prompt à la liste des phrases
        sentences.append(query)
        
        # Initialiser le vecteur TfidfVectorizer pour extraire les caractéristiques des phrases
        vectorizer = TfidfVectorizer()
        
        # Créer une matrice Tfidf à partir des phrases
        tfidf_matrix = vectorizer.fit_transform(sentences)
        
        # Convertir la matrice en liste de listes
        tfidf_list = tfidf_matrix.todense().tolist()
        
        # Créer un DataFrame pandas à partir de la liste de listes
        df = pd.DataFrame(tfidf_list, columns=vectorizer.get_feature_names())
        
        # Calculer la similarité cosinus entre le prompt et chaque phrase dans la matrice
        cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
        
        # Trier les similarités cosinus dans l'ordre décroissant
        similarity_list = cosine_similarities.flatten()
        index = similarity_list.argsort()[::-1]
        
        # Trouver l'entrée de corpus la plus similaire
        most_similar_sentence = sentences[index[0]+1]
        
        # Ajouter la réponse à la liste des phrases
        sentences.append(most_similar_sentence)
        
        # Afficher la réponse la plus similaire
        print(f"🤖 {str(most_similar_sentence)}")   
        

*********************************************************
Pour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays
Pour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet
Pour obtenir la définition d'un sujet, tapez : wiki + sujet
Pour quitter, tapez : quit ou exit
************************************************************
> cv


C:\Users\Mohamed-Amine-Miladi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


🤖 Cv hamdoullah
*********************************************************
Pour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays
Pour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet
Pour obtenir la définition d'un sujet, tapez : wiki + sujet
Pour quitter, tapez : quit ou exit
************************************************************
> chesmek


C:\Users\Mohamed-Amine-Miladi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


🤖 Esmi chatbot tounsi
*********************************************************
Pour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays
Pour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet
Pour obtenir la définition d'un sujet, tapez : wiki + sujet
Pour quitter, tapez : quit ou exit
************************************************************
> tcodi?


C:\Users\Mohamed-Amine-Miladi\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


🤖 Ey tawa hehda klem ?
*********************************************************
Pour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays
Pour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet
Pour obtenir la définition d'un sujet, tapez : wiki + sujet
Pour quitter, tapez : quit ou exit
************************************************************
> chnowa ta9s fi Tunis
ta9s: 🤖Fi Tunis aana temperature :22.9 °C wel humidité : 25%.
*********************************************************
Pour obtenir la météo d'un pays, tapez : chnowa ta9s fi + nom du pays
Pour obtenir les dernières actualités sur un sujet, tapez : e5er a5bar aala + sujet
Pour obtenir la définition d'un sujet, tapez : wiki + sujet
Pour quitter, tapez : quit ou exit
************************************************************
> wiki Abidjan
🤖نادي أسيك ميموسا (بالفرنسية: ASEC Mimosas)‏ هو نادي كرة قدم من ساحل العاج تأسس سنة 1948. يعتبر من بين النوادي الكبيرة على مستوى كرة القدم في